In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command,interrupt
from typing import  TypedDict
from langgraph.checkpoint.memory import MemorySaver


memory = MemorySaver()
class State(TypedDict):
    text: str

def node_a(state:State):
    print("node a")
    return Command(
        goto="node_b",
        update={
            "text": state["text"]+"a"
        }
    )
def node_b(state:State):
    print("node b")
    human_res = interrupt("do u want to go tp c or d: ")
    print(f"human response: {human_res}")
    if human_res=="c":
        return Command(
            goto="node_c",
            update={
                "text": state["text"]+"b"
            }
        )
    elif human_res=="d":
        return Command(
            goto="node_d",
            update={
                "text": state["text"]+"b"
            }
        )
def node_c(state: State):
    print("node_c")
    return Command(
        goto=END,
        update={
            "text": state["text"]+"c"
        }
    )
def node_d(state: State):
    print("node_d")
    return Command(
        goto=END,
        update={
            "text": state["text"]+"d"
        }
    )

graph = StateGraph(State)
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)
graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config  = {"configurable": {"thread_id":25}}

initial_state = {
    "text":""

}
first_res = app.invoke(initial_state, config, stream_mode="updates")
first_res


node a
node b


[{'node_a': {'text': 'a'}},
 {'__interrupt__': (Interrupt(value='do u want to go tp c or d: ', id='3b0f4feca64d65edf0c6fe51ff0c7890'),)}]

In [8]:
print(app.get_state(config).next)

('node_b',)


In [9]:
second_res = app.invoke(Command(resume="c"), config=config, stream_mode="updates")
second_res

node b
human response: c
node_c


[{'node_b': {'text': 'ab'}}, {'node_c': {'text': 'abc'}}]